In [1]:
from keras.layers import Conv2D,Input,Flatten,Layer,Dense,Reshape,Conv2DTranspose,Lambda,BatchNormalization,LeakyReLU,ReLU,Flatten,Dropout
from keras.models import Sequential,Model
from keras.backend import random_normal,exp
from keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy
from keras import layers
import keras.backend as K
import tensorflow as tf
import numpy as np
import random

In [2]:
class Convolution(Layer):
  def __init__(self,kernels,strides=2,Transpose = False,Activation="relu"):
    
    super().__init__()

    if(Transpose):
      self.Conv = Conv2DTranspose(kernels, 4,strides=strides, padding="same",use_bias=False)
    else:
      self.Conv = Conv2D(kernels, 4,strides=strides, padding="same",use_bias=False)

    self.Bn = BatchNormalization()
    self.act = ReLU() if Activation == "relu" else LeakyReLU(alpha=0.1)
  def call(self,X):

    y = self.Conv(X)
    y = self.Bn(y)
    y = self.act(y)

    return y

In [34]:
def Generator(DenseUnits,ConvUnits):

  Model = Sequential()
  Model.add(Input(shape=(100)))

  Model.add(Dense(DenseUnits[0],use_bias=False))
  Model.add(BatchNormalization())
  #Model.add(ReLU())
  
  
  

  Model.add(Dense(DenseUnits[1],use_bias=False))
  Model.add(BatchNormalization())
  #Model.add(ReLU())
  
  

  Model.add(Reshape((7,7,ConvUnits[0])))
  Model.add(Convolution(ConvUnits[1],Transpose=True))
  Model.add(Convolution(ConvUnits[2],Transpose=True))
  

  Model.add(Conv2DTranspose(ConvUnits[3], 1,strides=1, padding="same",activation="tanh"))

  return Model

In [41]:
# Classifier
def Classifier(ConvUnits):

  Model = Sequential()
  Model.add(Input(shape=(28,28,1)))
  Model.add(Convolution(ConvUnits[0],Activation="LeakyReLU"))
  Model.add(Dropout(0.1))
  Model.add(Convolution(ConvUnits[1],Activation="LeakyReLU"))
  Model.add(Dropout(0.1))
  Model.add(Flatten())
  
  Model.add(Dense(512, use_bias=False))
  Model.add(BatchNormalization())
  Model.add(LeakyReLU(alpha=0.1))


  return Model

In [42]:
DenseUnits = [1024,7*7*128]
ConvUnits_gen = [128,128,64,1]
ConvUnits_dis = [128,256]
ConvUnits_prox = [64,128]


Latent = {"noise":96,"features":4}

def GetVector(batch_size):

  noise = tf.random.uniform([batch_size, Latent["noise"]],-1.0,1.0)
  C = tf.random.uniform([batch_size, Latent["features"]],-1.0,1.0)
  Z_C = tf.concat([noise,C],axis=-1)

  return Z_C,C

In [43]:
from google.colab.patches import cv2_imshow

In [44]:

class InfoGan(Model):

  def __init__(self,use_same = False):
    super().__init__()

    self.use_same = use_same
    self.Generator = Generator(DenseUnits,ConvUnits_gen)
    
    if(use_same):
      self.Classifier = Classifier(ConvUnits_dis)
    else:
      self.Discriminator = Sequential([Classifier(ConvUnits_dis),Dense(2,use_bias=False,activation='softmax')])
      self.Proxy = Sequential([Classifier(ConvUnits_prox),Dense(128, use_bias=False),BatchNormalization(), LeakyReLU(alpha=0.1) ,Dense(Latent["features"],use_bias=False)])

  def compile(self,Gan=None,Disc=None,LossGan=None,LossDisc=None,LossProxy=None):

    super().compile()

    self.GanOpt = Gan
    self.DiscOpt = Disc
    self.LossGan = LossGan
    self.LossDisc = LossDisc
    self.LossProxy = LossProxy
  
  def CreateLabels(self,Batch_size,isGan):
    # Fake : 0
    # Original : 1
    if(isGan):
      return  tf.ones((Batch_size, 1))
    else:
      # X_all : [ Fake , Original ]
      return tf.concat([ tf.zeros((Batch_size, 1)) , tf.ones((Batch_size, 1)) ],axis=0)



  def train_step(self,Data):


    Batch_size = tf.shape(Data)[0]
    Z_C,C = GetVector(Batch_size)

    # Disc Training
    Labels = self.CreateLabels(Batch_size,False)
    X = self.Generator(Z_C)
   
    X_all =  tf.concat([X,Data],axis=0)

    with tf.GradientTape() as tape:

      Pred  = self.Discriminator(X_all)
      loss = self.LossDisc(Labels,Pred)
    
    grads = tape.gradient(loss, self.Discriminator.trainable_weights)
    self.DiscOpt.apply_gradients(zip(grads, self.Discriminator.trainable_weights))
    # Gen Training
    Z_C,C = GetVector(Batch_size*2)
    Labels = self.CreateLabels(Batch_size*2,True)
    
    with tf.GradientTape() as tape,tf.GradientTape() as tape2:

      X = self.Generator(Z_C)
      Pred = self.Discriminator(X)
      Prox = self.Proxy(X)

      loss_pred = self.LossGan(Labels,Pred)
      loss_prox = self.LossProxy(C,Prox)
      loss_gan = loss_pred + loss_prox
    
    

    

    grads = tape.gradient(loss_prox , self.Proxy.trainable_weights)
    self.GanOpt.apply_gradients(zip(grads,self.Proxy.trainable_weights ))

    grads = tape2.gradient(loss_gan , self.Generator.trainable_weights)
    self.GanOpt.apply_gradients(zip(grads,self.Generator.trainable_weights ))

    return {"d_loss":loss,"g_loss":loss_gan,"p_loss":loss_prox}

In [45]:
# LOSS Functions

def LossGan(true,pred):
  return  SparseCategoricalCrossentropy()(true,pred)
  
def LossDisc(true,pred):
  return SparseCategoricalCrossentropy()(true,pred)

def LossProxy(true,pred):
  #print(true.shape,pred.shape)
  return tf.reduce_mean(K.sum(K.square(true-pred),axis=(1)))


generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.05,clipnorm=1.0)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.05)

In [46]:
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = mnist_digits
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
print(mnist_digits.shape)

(70000, 28, 28, 1)


In [47]:
model = InfoGan()
model.compile(generator_optimizer, discriminator_optimizer, LossGan,LossDisc,LossProxy)
#model.fit(mnist_digits, batch_size=32, epochs=100)

In [48]:
import os 
#os.mkdir("./models")

In [49]:
for i in range(30):
  model.fit(mnist_digits, batch_size=128, epochs=1)
  model.save_weights('./models/infogan_'+str(i)+'.h5')
  print("epochs ",i)
  for j in range(10):
    Z_C,_ = GetVector(1)
    image = model.Generator(Z_C)
    score = model.Discriminator(image)
    pred = image.numpy().reshape(28,28)
    pred = (pred * 255).astype("int16")
    cv2_imshow(pred)
    print("Score : ",score.numpy())


547/547 [==============================] - 65s 114ms/step - d_loss: 0.6566 - g_loss: 1.0038 - p_loss: 0.2305
epochs  0


Score :  [[0.50206625 0.49793375]]


Score :  [[0.50335914 0.49664083]]


Score :  [[0.5710538 0.4289462]]


Score :  [[0.56804633 0.43195364]]


...

Score :  [[0.9965018  0.00349819]]


Score :  [[0.9921366  0.00786347]]


Score :  [[0.9942152  0.00578481]]


Score :  [[0.9950113  0.00498863]]


Score :  [[0.9977443  0.00225569]]


In [52]:
best_epoch_model = 15
model.built = True
model.load_weights('./models/infogan_'+str(best_epoch_model)+'.h5')


In [55]:

for i in range(10):
  print(" inference ",i)
  noise = tf.random.uniform([1, Latent["noise"]],-1.0,1.0)
  C = np.random.uniform(-1.0,1.0,[1,  Latent["features"]])

  for j in range(Latent["features"]):

    Cj = C.copy()
    Cj[0,j] = np.random.uniform(-1.0,1.0,[1])
    Z_C = tf.concat([noise,Cj],axis=-1)
    image = model.Generator(Z_C)
    pred = image.numpy().reshape(28,28)
    pred = (pred * 255).astype("int16")
    cv2_imshow(pred)
    
    #print(Cj,model.Proxy(image),LossProxy(Cj,model.Proxy(image)))


 inference  0


 inference  1


 inference  2


 inference  3


 inference  4


 inference  5


 inference  6


 inference  7


 inference  8


 inference  9
